In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np


from prototypes.CUHF2 import *
from prototypes.UNOCI import *
from prototypes.UHF import *
from prototypes.GHF import *
from prototypes.RHF import *
from prototypes.spin import *
from prototypes.optimization import *
from prototypes.FCI import *

## Parameters

In [2]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

basis = "6-31G"
N_H = 5
N_a = 3
N_b = 2

In [3]:
mol = gqcpy.Molecule.HChain(N_H, 1.4, 0)

In [4]:
_, HFpar = GHF(mol, basis)

GHF energy:  -2.6044402796004915


In [5]:
spinor_basis = gqcpy.GSpinorBasis_d(mol, basis)

## Testing

In [6]:
S_aa = spinor_basis.quantize(gqcpy.OverlapOperator()).parameters()[:spinor_basis.numberOfCoefficients(gqcpy.Spin.alpha), :spinor_basis.numberOfCoefficients(gqcpy.Spin.alpha)]

In [7]:
Sp = np.vstack((np.hstack((np.zeros_like(S_aa), S_aa)), np.hstack((np.zeros_like(S_aa), np.zeros_like(S_aa)))))

In [8]:
Sm = np.vstack((np.hstack((np.zeros_like(S_aa), np.zeros_like(S_aa))), np.hstack((S_aa, np.zeros_like(S_aa)))))

In [9]:
Spm = np.vstack((np.hstack((np.zeros_like(S_aa), np.zeros_like(S_aa))), np.hstack((np.zeros_like(S_aa), S_aa))))

In [10]:
Sz = spinor_basis.quantize(gqcpy.ElectronicSpin_zOperator()).parameters()

In [11]:
Sz2 = 0.25 * np.vstack((np.hstack((S_aa, np.zeros_like(S_aa))), np.hstack((np.zeros_like(S_aa), S_aa))))

In [12]:
spin = Sz2 + Sz + Spm
tensor = np.einsum('pq,rs->pqrs', Sz, Sz) + np.einsum('pq,rs->pqrs', Sm, Sp)

In [13]:
# mat_op = gqcpy.ScalarGSQOneElectronOperator_d(spin).transformed(HFpar.expansion())
# tensor_op = gqcpy.ScalarGSQTwoElectronOperator_d(tensor*2).transformed(HFpar.expansion())

mat_op = gqcpy.ScalarGSQOneElectronOperator_d(spin)
tensor_op = gqcpy.ScalarGSQTwoElectronOperator_d(tensor*2)

In [14]:
# OneDM = HFpar.calculateOrthonormalBasis1DM()
# twoDM = HFpar.calculateOrthonormalBasis2DM()

OneDM = HFpar.calculateScalarBasis1DM()
twoDM = HFpar.calculateScalarBasis2DM()

In [15]:
S2_val = mat_op.calculateExpectationValue(OneDM)[0] + tensor_op.calculateExpectationValue(twoDM)[0]
S2_val

0.7827024316525559

In [16]:
Sz_val = gqcpy.ScalarGSQOneElectronOperator_d(Sz).calculateExpectationValue(OneDM)[0]
Sz_val

0.4999999999999988

In [17]:
CUHF_object = CUHF2(mol, basis, N_a, N_b)
CUHF_object.SCF(mu=0, maxiter=15000)
print("CUHF energy: ", CUHF_object.energy + CUHF_object.nuc_rep)
print("CUHF S^2: ", CUHF_object.calculateSSquared())


converged!
CUHF energy:  -2.6044402691560027
CUHF S^2:  0.7827019561059161


In [19]:
S_2_op_product = spinor_basis.quantize(gqcpy.ElectronicSpinSquaredOperator())

In [20]:
S_2_op_product.calculateExpectationValue(OneDM, twoDM)

0.7827024316525559